In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import scipy.stats


def PCC_score(y_pred,y_true):
    diff_pred,diff_true=y_pred-np.mean(y_pred),y_true-np.mean(y_true)
    return np.sum(diff_pred*diff_true)/np.sqrt(np.sum(diff_pred**2)*np.sum(diff_true**2))

## 引用 MHC_flurry 文献的标准化代码
def from_ic50_limit_1(ic50, max_ic50=50000.0):
    """
    Convert ic50s to regression targets in the range [0.0, 1.0].
    
    Parameters
    ----------
    ic50 : numpy.array of float
    Returns
    -------
    numpy.array of float
    """
    x = 1.0 - (np.log(np.maximum(ic50, 1)) / np.log(max_ic50))
    
    return np.minimum(
        1.0,
        np.maximum(0.0, x))

def to_ic50(x, max_ic50=50000.0):
    return max_ic50 ** (1.0 - x)

def from_ic50_without_limit(ic50):
    x = 1.0 - (np.log(ic50))/np.log(50000)
    return x


def nmol_to_mol(ic50_nm):
    return ic50_nm * 1e-9

def negative_log10_ic50_mol(ic50_mol):
    x = -np.log10(ic50_mol)
    return x

def negative_log10_ic50_from_nm(ic50_nm):
    return negative_log10_ic50_mol(nmol_to_mol(ic50_nm))

测试函数功能正常

In [5]:
from_ic50_without_limit(0.17)
to_ic50(from_ic50_without_limit(0.17))
to_ic50(from_ic50_limit_1(0.17))
negative_log10_ic50_mol(1)
negative_log10_ic50_from_nm(1e9)

-0.0

new data net mhc pan

In [6]:
HLA_B_0702 = '/mnt/zt/Dopaap/calculate_webtools_score/calculate_hly_compare_task/HLA-B_0702_netMHCpan.csv'
HLA_A_3301 = '/mnt/zt/Dopaap/calculate_webtools_score/calculate_hly_compare_task/HLA-A_3301_netMHCpan.csv'
HLA_B_4403 = '/mnt/zt/Dopaap/calculate_webtools_score/calculate_hly_compare_task/HLA-B_4403_netMHCpan.csv'



df = pd.read_csv(HLA_B_0702)
df['netmhcpan_n'] = df['netmhcpan'].apply(from_ic50_without_limit)
df.to_csv(HLA_B_0702, index=0)
r2 = r2_score(df['Normalized_QM'].to_list(), df['netmhcpan_n'].to_list())
PCC = PCC_score(df['Normalized_QM'].to_list(), df['netmhcpan_n'].to_list())
tau = scipy.stats.kendalltau(df['Normalized_QM'].to_list(), df['netmhcpan_n'].to_list())[0]
print('HLA_B_0702','r2:', r2, 'PCC', PCC, 'tau', tau)

df = pd.read_csv(HLA_A_3301)
df['netmhcpan_n'] = df['netmhcpan'].apply(from_ic50_without_limit)
df.to_csv(HLA_A_3301, index=0)
r2 = r2_score(df['Normalized_QM'].to_list(), df['netmhcpan_n'].to_list())
PCC = PCC_score(df['Normalized_QM'].to_list(), df['netmhcpan_n'].to_list())
tau = scipy.stats.kendalltau(df['Normalized_QM'].to_list(), df['netmhcpan_n'].to_list())[0]
print('HLA_A_3301','r2:', r2, 'PCC', PCC, 'tau', tau)

df = pd.read_csv(HLA_B_4403)
df['netmhcpan_n'] = df['netmhcpan'].apply(from_ic50_without_limit)
df.to_csv(HLA_B_4403, index=0)
r2 = r2_score(df['Normalized_QM'].to_list(), df['netmhcpan_n'].to_list())
PCC = PCC_score(df['Normalized_QM'].to_list(), df['netmhcpan_n'].to_list())
tau = scipy.stats.kendalltau(df['Normalized_QM'].to_list(), df['netmhcpan_n'].to_list())[0]
print('HLA_B_4403','r2:', r2, 'PCC', PCC, 'tau', tau)

HLA_B_0702 r2: -0.018897350257493395 PCC 0.6168394394433353 tau 0.46798029556650234
HLA_A_3301 r2: 0.8845057226464234 PCC 0.9862749175059198 tau 0.6666666666666669


FileNotFoundError: [Errno 2] No such file or directory: '/raid/zt/MHC_IC50_regression/calculate_hly_compare_task/HLA-A_4403_netMHCpan.csv'

In [19]:
full_allele_for_NMp = '/mnt/zt/Dopaap/calculate_webtools_score/calculate_hly_compare_task/cal_peptide_prediction_lasted.csv'
except_b2706_for_not_full = '/mnt/zt/Dopaap/calculate_webtools_score/calculate_hly_compare_task/cal_not_full.csv'
our_limit_1 = '/mnt/zt/Dopaap/calculate_webtools_score/calculate_hly_compare_task/PK3-predresults_noMS_V1.csv'


df_full = pd.read_csv(full_allele_for_NMp)
df_not_full = pd.read_csv(except_b2706_for_not_full)
df_our_limit = pd.read_csv(our_limit_1)


# 数据处理  
# N1:对以nmol为单位的ic50值进行限制[0, 1]范围的MHC_flurry标准化
# N2:对以nmol为单位的ic50值进行不限制范围的MHC_flurry标准化
# n_log10:把ic50值转化为mol为单位后, 取以10为底的负对数
df_full['QM_N1'] = df_full['QM'].apply(from_ic50_limit_1)
df_full['QM_N2'] = df_full['QM'].apply(from_ic50_without_limit)

df_full['NetMHCpan_N1'] = df_full['NetMHCpan'].apply(from_ic50_limit_1)
df_full['NetMHCpan_N2'] = df_full['NetMHCpan'].apply(from_ic50_without_limit)

df_full['QM_n_log10'] = df_full['QM'].apply(negative_log10_ic50_from_nm)
df_full['pred_QM_n_log10'] = df_full['pred_QM'].apply(negative_log10_ic50_from_nm)
df_full['NetMHCpan_n_log10'] = df_full['NetMHCpan'].apply(negative_log10_ic50_from_nm)

df_not_full['QM_N1'] = df_not_full['QM'].apply(from_ic50_limit_1)
df_not_full['QM_N2'] = df_not_full['QM'].apply(from_ic50_without_limit)

df_not_full['SMM_N1'] = df_not_full['SMM'].apply(from_ic50_limit_1)
df_not_full['SMM_N2'] = df_not_full['SMM'].apply(from_ic50_without_limit)
df_not_full['SMMPMBEC_N1'] = df_not_full['SMMPMBEC'].apply(from_ic50_limit_1)
df_not_full['SMMPMBEC_N2'] = df_not_full['SMMPMBEC'].apply(from_ic50_without_limit)
df_not_full['ANN4_N1'] = df_not_full['ANN4'].apply(from_ic50_limit_1)
df_not_full['ANN4_N2'] = df_not_full['ANN4'].apply(from_ic50_without_limit)
df_not_full['NetMHCpan_N1'] = df_not_full['NetMHCpan'].apply(from_ic50_limit_1)
df_not_full['NetMHCpan_N2'] = df_not_full['NetMHCpan'].apply(from_ic50_without_limit)

df_not_full['QM_n_log10'] = df_not_full['QM'].apply(negative_log10_ic50_from_nm)
df_not_full['pred_QM_n_log10'] = df_not_full['pred_QM'].apply(negative_log10_ic50_from_nm)
df_not_full['SMM_n_log10'] = df_not_full['SMM'].apply(negative_log10_ic50_from_nm)
df_not_full['SMMPMBEC_n_log10'] = df_not_full['SMMPMBEC'].apply(negative_log10_ic50_from_nm)
df_not_full['ANN4_n_log10'] = df_not_full['ANN4'].apply(negative_log10_ic50_from_nm)
df_not_full['NetMHCpan_n_log10'] = df_not_full['NetMHCpan'].apply(negative_log10_ic50_from_nm)

df_our_limit['QM_N1'] = df_our_limit['QM'].apply(from_ic50_limit_1)
df_our_limit['QM_N2'] = df_our_limit['QM'].apply(from_ic50_without_limit)
df_our_limit['QM_n_log10'] = df_our_limit['QM'].apply(negative_log10_ic50_from_nm)
df_our_limit['pred_QM_n_log10'] = df_our_limit['pred_QM'].apply(negative_log10_ic50_from_nm)



# 保存文件
df_full.to_csv(full_allele_for_NMp, index=0)
df_not_full.to_csv(except_b2706_for_not_full, index=0)
df_our_limit.to_csv(our_limit_1, index=0)

In [20]:
def output_result(y_true, tool_name_list, tool_data_list):
    for i in range(len(tool_data_list)):
        MAE = mean_absolute_error(y_true, tool_data_list[i])
        MSE = mean_squared_error(y_true, tool_data_list[i])
        RMSE = np.sqrt(MSE)
        r2 = r2_score(y_true, tool_data_list[i])
        PCC = PCC_score(y_true, tool_data_list[i])
        tau = scipy.stats.kendalltau(y_true, tool_data_list[i])[0]
        print(f'{tool_name_list[i]}', 'MAE', MAE, 'MSE', MSE, 'RMSE', RMSE, 'r2:', r2, 'PCC', PCC, 'tau', tau)

实验测定值直接计算, 单位为nm

In [21]:
print('实验测定值直接计算, 单位为nm')
# 无allele hla-b_2706
QM_not_full = df_not_full['QM'].to_list()
SMM = df_not_full["SMM"].to_list()
SMMP = df_not_full["SMMPMBEC"].to_list()
ANN = df_not_full["ANN4"].to_list()
NetMHC_less = df_not_full["NetMHCpan"].to_list()
pred_QM_less = df_not_full["pred_QM"].to_list()

tool_name_list = ['Dopaap', 'SMM', 'SMMP', 'ANN', 'NetMHC']
tool_data_list = [pred_QM_less, SMM, SMMP, ANN, NetMHC_less]
print('\n无allele hla-b_2706')
output_result(QM_not_full, tool_name_list, tool_data_list)

# 全32allele, dopaap_pred from test in train 未限制0-1 进行还原
QM_full = df_full['QM'].to_list()
NetMHC_full = df_full["NetMHCpan"].to_list()
pred_QM_nl = df_full["pred_QM"].to_list() # not limit

tool_name_list = ['Dopaap', 'NetMHC']
tool_data_list = [pred_QM_nl, NetMHC_full]
print('\n全32allele, dopaap_pred from test in train 未限制0-1 进行还原')
output_result(QM_full, tool_name_list, tool_data_list)


# our最新数据  限制0-1还原
QM_our = df_our_limit['QM'].to_list()
pred_QM = df_our_limit["pred_QM"].to_list()

tool_name_list = ['Dopaap']
tool_data_list = [pred_QM]
print('\nour最新数据  限制0-1还原')
output_result(QM_our, tool_name_list, tool_data_list)

实验测定值直接计算, 单位为nm

无allele hla-b_2706
Dopaap MAE 2850.1412234948834 MSE 45383677.092442185 RMSE 6736.740836075126 r2: 0.5748218906780134 PCC 0.7749167409409363 tau 0.7158811254065496
SMM MAE 31082.787356096807 MSE 65695367353.18124 RMSE 256311.07536191493 r2: -614.468685482309 PCC 0.1056776602228501 tau 0.5067720774780522
SMMP MAE 26341.40396231995 MSE 30830660489.643353 RMSE 175586.61819638577 r2: -287.8378107713449 PCC 0.10485235069329991 tau 0.5078549695074709
ANN MAE 6882.010587463557 MSE 148605151.56519273 RMSE 12190.371264452642 r2: -0.39221106410784845 PCC 0.3068478634844825 tau 0.5134061726883399
NetMHC MAE 6201.2432434402335 MSE 126557509.84004578 RMSE 11249.778212926945 r2: -0.18565718341166249 PCC 0.3594925225357332 tau 0.542462066925058

全32allele, dopaap_pred from test in train 未限制0-1 进行还原
Dopaap MAE 2862.1719058978633 MSE 45796650.89126619 RMSE 6767.322283685489 r2: 0.5727720809500537 PCC 0.7740399922814678 tau 0.7200882841133374
NetMHC MAE 6168.231581181464 MSE 124965023.

1-log50000(x), 限制[0,1]范围计算, x单位为nm

In [22]:
print('1-log50000(x), 限制[0,1]范围计算, x单位为nm')
# 无allele hla-b_2706
QM_not_full = df_not_full['QM_N1'].to_list()
SMM = df_not_full["SMM_N1"].to_list()
SMMP = df_not_full["SMMPMBEC_N1"].to_list()
ANN = df_not_full["ANN4_N1"].to_list()
NetMHC_less = df_not_full["NetMHCpan_N1"].to_list()
pred_QM_less = df_not_full["P_N_QM"].to_list()

tool_name_list = ['Dopaap', 'SMM', 'SMMP', 'ANN', 'NetMHC']
tool_data_list = [pred_QM_less, SMM, SMMP, ANN, NetMHC_less]
print('\n无allele hla-b_2706')
output_result(QM_not_full, tool_name_list, tool_data_list)

# 全32allele, dopaap_pred from test in train 未限制0-1 进行还原
QM_full = df_full['QM_N1'].to_list()
NetMHC_full = df_full["NetMHCpan_N1"].to_list()
pred_QM_nl = df_full["P_N_QM"].to_list() # not limit

tool_name_list = ['Dopaap', 'NetMHC']
tool_data_list = [pred_QM_nl, NetMHC_full]
print('\n全32allele, dopaap_pred from test in train 未限制0-1 进行还原')
output_result(QM_full, tool_name_list, tool_data_list)


# our最新数据  限制0-1还原
QM_our = df_our_limit['QM_N1'].to_list()
pred_QM = df_our_limit["P_N_QM"].to_list()

tool_name_list = ['Dopaap']
tool_data_list = [pred_QM]
print('\nour最新数据  限制0-1还原')
output_result(QM_our, tool_name_list, tool_data_list)

1-log50000(x), 限制[0,1]范围计算, x单位为nm

无allele hla-b_2706
Dopaap MAE 0.09371570394531967 MSE 0.020644547534760742 RMSE 0.14368210582658072 r2: 0.854101991067867 PCC 0.9278127280270093 tau 0.7592334801067653
SMM MAE 0.2283125108278742 MSE 0.07722406398245796 RMSE 0.27789218049894454 r2: 0.4542463496612139 PCC 0.7265512881383963 tau 0.5257592591462927
SMMP MAE 0.22761350578206652 MSE 0.07635124616267203 RMSE 0.27631729255092236 r2: 0.4604146796695532 PCC 0.7290190235942223 tau 0.5273712552742171
ANN MAE 0.2048430274902422 MSE 0.06524295155821878 RMSE 0.25542699849119077 r2: 0.5389186072897847 PCC 0.7734683358127291 tau 0.533803975080695
NetMHC MAE 0.19140161353171745 MSE 0.05731503776049783 RMSE 0.2394055925840034 r2: 0.5949463228948619 PCC 0.8064709866232317 tau 0.5659402339057924

全32allele, dopaap_pred from test in train 未限制0-1 进行还原
Dopaap MAE 0.09230821340791 MSE 0.020189597897413623 RMSE 0.14209010485397505 r2: 0.8560796449404279 PCC 0.9286744697039431 tau 0.7609647682173719
NetMHC MAE

1-log50000(x), 不限制[0,1]范围计算, x单位为nm

In [23]:
print('1-log50000(x), 不限制[0,1]范围计算, x单位为nm')
# 无allele hla-b_2706
QM_not_full = df_not_full['QM_N2'].to_list()
SMM = df_not_full["SMM_N2"].to_list()
SMMP = df_not_full["SMMPMBEC_N2"].to_list()
ANN = df_not_full["ANN4_N2"].to_list()
NetMHC_less = df_not_full["NetMHCpan_N2"].to_list()
pred_QM_less = df_not_full["P_N_QM"].to_list()

tool_name_list = ['Dopaap', 'SMM', 'SMMP', 'ANN', 'NetMHC']
tool_data_list = [pred_QM_less, SMM, SMMP, ANN, NetMHC_less]
print('\n无allele hla-b_2706')
output_result(QM_not_full, tool_name_list, tool_data_list)

# 全32allele, dopaap_pred from test in train 未限制0-1 进行还原
QM_full = df_full['QM_N2'].to_list()
NetMHC_full = df_full["NetMHCpan_N2"].to_list()
pred_QM_nl = df_full["P_N_QM"].to_list() # not limit

tool_name_list = ['Dopaap', 'NetMHC']
tool_data_list = [pred_QM_nl, NetMHC_full]
print('\n全32allele, dopaap_pred from test in train 未限制0-1 进行还原')
output_result(QM_full, tool_name_list, tool_data_list)


# our最新数据  限制0-1还原
QM_our = df_our_limit['QM_N2'].to_list()
pred_QM = df_our_limit["P_N_QM"].to_list()

tool_name_list = ['Dopaap']
tool_data_list = [pred_QM]
print('\nour最新数据  限制0-1还原')
output_result(QM_our, tool_name_list, tool_data_list)

1-log50000(x), 不限制[0,1]范围计算, x单位为nm

无allele hla-b_2706
Dopaap MAE 0.1257235483156042 MSE 0.03234000826213124 RMSE 0.1798332790729548 r2: 0.8177834154518069 PCC 0.9259041737033104 tau 0.7158811254065496
SMM MAE 0.2707505817124533 MSE 0.10561227027632318 RMSE 0.3249804152196301 r2: 0.4049380872092563 PCC 0.7236010436112371 tau 0.5067720774780522
SMMP MAE 0.269591505903248 MSE 0.10432769691107459 RMSE 0.32299798282818204 r2: 0.41217588904653246 PCC 0.7264938598816544 tau 0.5078549695074709
ANN MAE 0.23950318384877983 MSE 0.08516079420134168 RMSE 0.291823224232311 r2: 0.5201699105639801 PCC 0.7827929436921278 tau 0.5134061726883399
NetMHC MAE 0.2260617698902551 MSE 0.0770018437918941 RMSE 0.2774920607727257 r2: 0.5661407113460095 PCC 0.813599102322912 tau 0.542462066925058

全32allele, dopaap_pred from test in train 未限制0-1 进行还原
Dopaap MAE 0.12337631561946315 MSE 0.03154168287650972 RMSE 0.1775997828729239 r2: 0.8205868130742409 PCC 0.9267464307879215 tau 0.7200882841133374
NetMHC MAE 0.221

-log10(x)计算, x单位为mol

In [24]:
print('-log10(x)计算, x单位为mol')
# 无allele hla-b_2706
QM_not_full = df_not_full['QM_n_log10'].to_list()
SMM = df_not_full["SMM_n_log10"].to_list()
SMMP = df_not_full["SMMPMBEC_n_log10"].to_list()
ANN = df_not_full["ANN4_n_log10"].to_list()
NetMHC_less = df_not_full["NetMHCpan_n_log10"].to_list()
pred_QM_less = df_not_full["pred_QM_n_log10"].to_list()

tool_name_list = ['Dopaap', 'SMM', 'SMMP', 'ANN', 'NetMHC']
tool_data_list = [pred_QM_less, SMM, SMMP, ANN, NetMHC_less]
print('\n无allele hla-b_2706')
output_result(QM_not_full, tool_name_list, tool_data_list)

# 全32allele, dopaap_pred from test in train 未限制0-1 进行还原
QM_full = df_full['QM_n_log10'].to_list()
NetMHC_full = df_full["NetMHCpan_n_log10"].to_list()
pred_QM_nl = df_full["pred_QM_n_log10"].to_list() # not limit

tool_name_list = ['Dopaap', 'NetMHC']
tool_data_list = [pred_QM_nl, NetMHC_full]
print('\n全32allele, dopaap_pred from test in train 未限制0-1 进行还原')
output_result(QM_full, tool_name_list, tool_data_list)


# our最新数据  限制0-1还原
QM_our = df_our_limit['QM_n_log10'].to_list()
pred_QM = df_our_limit["pred_QM_n_log10"].to_list()

tool_name_list = ['Dopaap']
tool_data_list = [pred_QM]
print('\nour最新数据  限制0-1还原')
output_result(QM_our, tool_name_list, tool_data_list)

-log10(x)计算, x单位为mol

无allele hla-b_2706
Dopaap MAE 0.5907711823733203 MSE 0.7140777021796086 RMSE 0.8450311841462471 r2: 0.8177834154513565 PCC 0.9259041737031188 tau 0.7158811254065496
SMM MAE 1.2722488621233465 MSE 2.331952628750883 RMSE 1.5270732231137063 r2: 0.4049380872092563 PCC 0.7236010436112369 tau 0.5067720774780522
SMMP MAE 1.2668023996631392 MSE 2.3035888389367147 RMSE 1.517757832770668 r2: 0.41217588904653224 PCC 0.7264938598816545 tau 0.5078549695074709
ANN MAE 1.1254182768483914 MSE 1.8803775109155387 RMSE 1.3712685772362534 r2: 0.5201699105639801 PCC 0.7827929436921279 tau 0.5134061726883399
NetMHC MAE 1.06225747584142 MSE 1.7002252823404014 RMSE 1.3039268700124258 r2: 0.5661407113460095 PCC 0.8135991023229121 tau 0.542462066925058

全32allele, dopaap_pred from test in train 未限制0-1 进行还原
Dopaap MAE 0.5797416063407521 MSE 0.6964504229179344 RMSE 0.8345360524973947 r2: 0.8205868130738387 PCC 0.9267464307877648 tau 0.7200882841133374
NetMHC MAE 1.0416361838288306 MSE 1.6563